In [3]:
import json
with open('/Users/ramz.sivagurunathan/hacks/hackathon/repos/transport-analyser/data/interchangeInfo.json', 'rb') as fp:
    interchangeInfo = json.load(fp)

In [25]:
buses_list = []
for k,v in interchangeInfo.iteritems():
    buses_list.extend(v['buses'])
buses_list = set(buses_list)

#### buses = {trips: [{route_id: xxx, route_name: xxx, service_id: xxx, start_date: xxx, end_date: xxx , operating_days: [], time_start: xxx , time_end: xxx, total_time: xxx, stops: [id: xxx, lat: xxx, lon: xxx, dist: xxx]}] }

In [45]:
import csv
bus_trips = {}
route_id_list = set({})
trip_id_list = set({})
for bus in buses_list:
    bus_trips[bus] = {'trips': []}
with open('/Users/ramz.sivagurunathan/hacks/hackathon/datasets/routes/trips.txt','r') as fp:
    reader = csv.reader(fp)
    for row in reader:
        if reader.line_num > 1:
            bus = row[2].split('-')[1]
            trip_id = row[2]
            route_id = row[0]
            service_id = row[1]
            direction = row[5]
            head_sign = row[4]
            wheelchair = row[7]
            if bus in buses_list:
                trip_ids = map(lambda x: x['id'], bus_trips[bus]['trips'])
                if trip_id not in trip_ids:
                    bus_trips[bus]['trips'].append({'id' : trip_id, 'route_id': route_id, 
                                                    'service_id': service_id, 'direction' : direction, 
                                                    'head_sign' : head_sign, 'wheelchair': wheelchair})
                    route_id_list.add(route_id)
                    trip_id_list.add(trip_id)

In [38]:
with open('/Users/ramz.sivagurunathan/hacks/hackathon/datasets/routes/routes.txt','r') as fp:
    reader = csv.reader(fp)
    for row in reader:
        for idx, bus_trip in bus_trips.iteritems():
            for trip_idx, trip_info in enumerate(bus_trip['trips']):
                if row[0] == trip_info['route_id']:
                    bus_trips[idx]['trips'][trip_idx]['route_name'] = row[3]
print bus_trips['M40']

{'trips': [{'direction': '0', 'wheelchair': '1', 'route_name': 'Prepay Only - Chatswood to Bondi Junction', 'head_sign': 'Chatswood', 'route_id': '90-M40-sj2-1', 'service_id': 'b00ou+48', 'id': '327.b00ou.90-M40-sj2-1.1.H'}, {'direction': '0', 'wheelchair': '1', 'route_name': 'Prepay Only - Chatswood to Bondi Junction', 'head_sign': 'Chatswood', 'route_id': '90-M40-sj2-1', 'service_id': 'b00ou+48', 'id': '329.b00ou.90-M40-sj2-1.1.H'}, {'direction': '0', 'wheelchair': '1', 'route_name': 'Prepay Only - Chatswood to Bondi Junction', 'head_sign': 'Chatswood', 'route_id': '90-M40-sj2-1', 'service_id': 'b00ou+48', 'id': '343.b00ou.90-M40-sj2-1.1.H'}, {'direction': '0', 'wheelchair': '1', 'route_name': 'Prepay Only - Chatswood to Bondi Junction', 'head_sign': 'Chatswood', 'route_id': '90-M40-sj2-1', 'service_id': 'b00ou+48', 'id': '349.b00ou.90-M40-sj2-1.1.H'}, {'direction': '0', 'wheelchair': '1', 'route_name': 'Prepay Only - Chatswood to Bondi Junction', 'head_sign': 'Chatswood', 'route_id'

In [39]:
for bus, bus_info in bus_trips.iteritems():
    with open("/Users/ramz.sivagurunathan/hacks/hackathon/repos/transport-analyser/data/buses/{0}.json".format(bus), 'w') as fp:
        json.dump(bus_info, fp, indent=4)

In [52]:
for idx, bus_trip in bus_trips.iteritems():
    trip_id_list = map(lambda x: x['id'], bus_trip['trips'])
    with open('/Users/ramz.sivagurunathan/hacks/hackathon/datasets/routes/stop_times.txt','r') as fp:
        reader = csv.reader(fp)
        for row in reader:
            if row[0] in trip_id_list:
                for trip_idx, trip_info in enumerate(bus_trip['trips']):
                    if row[0] == trip_info['id']:
                        bus_trips[idx]['trips'][trip_idx]['stops'] = bus_trips[idx]['trips'][trip_idx].get('stops', [])
                        if len(bus_trips[idx]['trips'][trip_idx]['stops']) + 1 == int(row[4]):
                            bus_trips[idx]['trips'][trip_idx]['stops'].append(row[3])
    with open("/Users/ramz.sivagurunathan/hacks/hackathon/repos/transport-analyser/data/buses/{0}.json".format(idx), 'w') as fp:
        json.dump(bus_trips[idx], fp, indent=4)
        print("Done for {0}".format(idx))
        

Done for T72
Done for T70
Done for T71
Done for T74
Done for T75
Done for 818
Done for 819
Done for 343
Done for 810
Done for 811
Done for 812
Done for 813
Done for 348
Done for 815
Done for 816
Done for 817
Done for 718
Done for 715
Done for 714
Done for 711
Done for 3SH
Done for 294
Done for 292
Done for 290
Done for T80
Done for 3ES
Done for 195
Done for 197
Done for 196
Done for 271
Done for 275
Done for 277
Done for 278
Done for 279
Done for 524
Done for 525
Done for 520
Done for 521
Done for 523
Done for 10T
Done for 440
Done for 100
Done for 106
Done for 107
Done for 104
Done for 902
Done for 903
Done for 900
Done for 901
Done for 906
Done for 907
Done for 904
Done for 905
Done for 33
Done for 32
Done for 908
Done for 909
Done for 37
Done for 36
Done for 34
Done for 642
Done for 644
Done for 334
Done for S1
Done for S7
Done for 333
Done for S4
Done for 558
Done for 854
Done for 855
Done for 856
Done for 857
Done for 850
Done for 851
Done for 852
Done for 853
Done for 546
Done fo

In [65]:
from datetime import datetime

calendar_details = {}
with open('/Users/ramz.sivagurunathan/hacks/hackathon/datasets/routes/calendar.txt', 'r') as fp:
    reader = csv.reader(fp)
    for row in reader:
        if reader.line_num > 1:
            calendar_details[row[0]] = {'operating_days': [row[1], row[2], row[3], row[4], row[5], row[6], row[7]],
                                        'start_date': row[8], 'end_date': row[9]
                                       }
for bus in bus_trips.keys():
    with open("/Users/ramz.sivagurunathan/hacks/hackathon/repos/transport-analyser/data/buses/{0}.json".format(bus), 'r') as  fp:
        bus_info = json.load(fp)
        for idx, trip in enumerate(bus_info['trips']):
            trip_updated = {}
            trip_updated['service_id'] = trip['service_id']
            trip_updated['stops'] = trip['stops']
            trip_updated['id'] = trip['id']
            trip_updated['route_id'] = trip['route_id']
            trip_updated['direction'] = trip['direction']
            trip_updated['head_sign'] = trip['head_sign']
            trip_updated['wheelchair'] = trip['wheelchair']
            
            service_info = calendar_details[trip['service_id']]
            trip_updated['operating_days'] = service_info['operating_days']
            # Date is dumped in ISO 8601 format
            trip_updated['start_date'] = datetime.strptime(service_info['start_date'], '%Y%m%d').isoformat()
            trip_updated['end_date'] = datetime.strptime(service_info['end_date'], '%Y%m%d').isoformat()
            bus_info['trips'][idx] = trip_updated
    with open("/Users/ramz.sivagurunathan/hacks/hackathon/repos/transport-analyser/data/buses/{0}.json".format(bus), 'w') as  fp:
        json.dump(bus_info, fp)


In [64]:
from datetime import datetime

stop_details = {}
with open('/Users/ramz.sivagurunathan/hacks/hackathon/datasets/routes/stops.txt', 'r') as fp:
    reader = csv.reader(fp)
    for row in reader:
        if reader.line_num > 1:
            stop_details[row[0]] = { 'name' : row[2], 'lat' : row[3], 'lon' : row[4], 'wheelchair' : row[7] }

for bus in bus_trips.keys():
    with open("/Users/ramz.sivagurunathan/hacks/hackathon/repos/transport-analyser/data/buses/{0}.json".format(bus), 'r') as  fp:
        bus_info = json.load(fp)
        for trip in bus_info['trips']:
            stop_info = []
            for stop in trip['stops'] :
                stop_info.append({ 'id' : stop, 'name' : stop_details[stop]['name'], 'lat' : stop_details[stop]['lat'],
                                  'lon' : stop_details[stop]['lon'], 'wheelchair' : stop_details[stop]['wheelchair']
                                 })
            trip['stops'] = stop_info
    with open("/Users/ramz.sivagurunathan/hacks/hackathon/repos/transport-analyser/data/buses/{0}.json".format(bus), 'w') as  fp:
        json.dump(bus_info, fp)

In [66]:
for bus in bus_trips.keys():
    with open("/Users/ramz.sivagurunathan/hacks/hackathon/repos/transport-analyser/data/buses/{0}.json".format(bus), 'r') as  fp:
        bus_info = json.load(fp)
    with open("/Users/ramz.sivagurunathan/hacks/hackathon/repos/transport-analyser/data/buses/{0}.json".format(bus), 'w') as  fp:
        json.dump(bus_info, fp, indent=4)

In [85]:
trip_start_end_time = {}
with open('/Users/ramz.sivagurunathan/hacks/hackathon/datasets/routes/stop_times.txt', 'r') as fp:
    reader = csv.reader(fp)
    for row in reader:
        if reader.line_num > 1:
            try:
                if row[4] == "1":
                    trip_start_end_time[row[0]] = {'start_time' : datetime.strptime(row[1], "%H:%M:%S").isoformat()}
                else:
                    trip_start_end_time[row[0]]['end_time'] = datetime.strptime(row[2], "%H:%M:%S").isoformat()
            except:
                pass

In [88]:
for bus in bus_trips.keys():
    print bus
    with open("/Users/ramz.sivagurunathan/hacks/hackathon/repos/transport-analyser/data/buses/{0}.json".format(bus), 'r') as  fp:
        bus_info = json.load(fp)
        for idx, trip in enumerate(bus_info['trips']):
            try:
                bus_info['trips'][idx]['start_time'] = trip_start_end_time[trip['id']]['start_time']
                bus_info['trips'][idx]['end_time'] = trip_start_end_time[trip['id']]['end_time']
            except:
                pass
    with open("/Users/ramz.sivagurunathan/hacks/hackathon/repos/transport-analyser/data/buses/{0}.json".format(bus), 'w') as  fp:
        json.dump(bus_info, fp, indent=4)

T72


ValueError: No JSON object could be decoded